# Browser Automation Lecture Notes
Date: 2025-07-09

We'll use this notebook to work through some examples and showcase some essential functions in Playwright.

In [1]:
!pip install ipykernel==6.28.0

In [9]:
import os
import random
import time

from playwright.async_api import async_playwright, expect

In [3]:
!pip install playwright

In [4]:
!which playwright

/Users/lyin72/miniconda3/bin/playwright


In [5]:
!playwright install

In [10]:
os.makedirs('data/', exist_ok=True)

In [11]:
# Start the browser
playwright = await async_playwright().start()

In [13]:
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()
# await browser.close()

In [14]:
await browser.close()

In [15]:
async def open_browser(headless=False):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open chromium (chrome) browser, can use firefox or others
    browser = await playwright.chromium.launch(headless=headless)
  
    # Create a new browser window
    page = await browser.new_page()

    return browser, page

In [49]:
driver, page = await open_browser()

In [50]:
# visit a URL
url = 'https://amazon.com'
await page.goto(url)

<Response url='https://www.amazon.com/' request=<Request url='https://www.amazon.com/' method='GET'>>

## XPATH
You learned about beautifulSoup to work with HTML...

Xpath is another way to navigate hierarchical structures of HTML and SVG

It's fast, versatile, and can be used in the developer console and in any computing language.

## Using XPATH
You can test xpaths in the devloper tools under the `console` tab using the function `$x()`. Read about that function [here](https://developer.chrome.com/docs/devtools/console/utilities/#xpath-function).


You can use Playwright's `locator` [function](https://playwright.dev/python/docs/locators#locate-by-css-or-xpath) to find the search box on Amazon site using an xpath or css selector. It'll return the first match unless you add `.all()` to the located elements.

Playwright allows other [locators](https://playwright.dev/python/docs/locators#quick-guide), which the developers suggest (but I don't).

Example of an xpath!

//input[@aria-label="Search Amazon"]

### Finding elements and performing actons
Browser automation is largely about locating elements on the page, and interacting with them in some way.
This can involve filling forms, mocking pressing buttons on a keyboard, clicking things.

In [51]:
# here's how you can do that with xpath
search_bar = page.locator(
    '//input[@aria-label="Search Amazon"]'
)

In [22]:
search_bar

<Locator frame=<Frame name= url='https://www.amazon.com/'> selector='//input[@aria-label="Search Amazon"]'>

You can also use the built-in functionality by exploiting the placeholder text
```
page.get_by_placeholder("Search Amazon")
```

Let's fill in the search bar.

In [23]:
await search_bar.fill('TEST')

In [52]:
search_term = 'cool shirt'
await search_bar.fill(search_term)

You can make the search either by inputting the "Enter" button, or finding the search button and clicking it.

Here's how you can press a [keyboard](https://playwright.dev/docs/api/class-keyboard) button.

In [53]:
await page.keyboard.press("Enter")

Alternatively, you can locate the button perform an [action](https://playwright.dev/python/docs/input#mouse-click) such as a mouse `click`.

In [27]:
search_button = page.locator(
    '//input[@id="nav-search-submit-button"]'
)
await search_button.click()

### Parse the products

For each product, let's print the brand name:

In [36]:
# contains supports substring matches
xpath_product = '//div[contains(@cel_widget_id, "MAIN-SEARCH_RESULTS-")]'
product_tiles = await page.locator(xpath_product).all()
len(product_tiles)

/var/folders/sj/0bmsn0t57yb1tzg0xyg_9thm0000gq/T/ipykernel_96454/988507552.py:3: RuntimeWarning: coroutine 'Locator.all' was never awaited
  product_tiles = await page.locator(xpath_product).all()


60

In [54]:
# same as above, but does not use substring match
xpath_product = '//div[@data-component-type="s-search-result"]'
product_tiles = await page.locator(xpath_product).all()
len(product_tiles)

60

In [39]:
product_tiles[0]

<Locator frame=<Frame name= url='https://www.amazon.com/s?k=cool+shirt&crid=SQDCPMTPLQL2&sprefix=cool+shirt%2Caps%2C110&ref=nb_sb_noss_1'> selector='//div[@data-component-type="s-search-result"] >> nth=0'>

In [34]:
# this is what it looks like to get ONE thing (always the first)
product_tile = page.locator(xpath_product)
product_tile

<Locator frame=<Frame name= url='https://www.amazon.com/s?k=cool+shirt&crid=SQDCPMTPLQL2&sprefix=cool+shirt%2Caps%2C110&ref=nb_sb_noss_1'> selector='//div[@data-component-type="s-search-result"]'>

Notice we're adding `.all()` to the command, this will return a list, rather than the first element.

If you run all the cells at once, the above will return zero results. This is because the browser needs to await for the element to be visible.

You can force the browser to sleep or check the element is visible using the `expect` [function](https://playwright.dev/python/docs/test-assertions).

In [40]:
# Let's wait for the first product to load
await expect(page.locator(xpath_product).first).to_be_visible()

In [41]:
# run it again, after waiting for the elements to be rendered
product_tiles = await page.locator(xpath_product).all()
len(product_tiles)

60

This is the sleeping method

In [22]:
# import asyncio

# await asyncio.sleep(2)
# product_tiles = await page.locator(xpath_product).all()

Let's parse one product

In [42]:
prod = product_tiles[0]

In [ ]:
prod

In [43]:
# see the text of the element
await prod.text_content()

'\n\n\n\n    \n\n\n\n    +9 other colors/patternsFeatured from Amazon brandsFeatured from Amazon brands Amazon EssentialsSlim-Fit Long Sleeve Shirt for Men, Pocket and No Pocket Styles 4.3 out of 5 stars 7,882  50+ bought in past monthPrime Day DealPrice, product page$9.68$9.68 Typical price: $12.30Typical price: $12.30$12.30Exclusive Prime priceFREE delivery Mon, Jul 14 on $35 of items shipped by AmazonOr fastest delivery Tomorrow, Jul 10  1 sustainability feature<img alt="" src="https://m.media-amazon.com/images/I/11++B3A2NEL.png" height="24px" width="24px"/>  Sustainability featuresThis product has sustainability features recognized by trusted certifications. Safer chemicalsMade with chemicals safer for human health and the environment.As certified by<img alt="" src="https://m.media-amazon.com/images/I/51YvKwF01yL._SS200_.jpg" height="24px" width="24px"/>  OEKO-TEX STANDARD 100Learn more about OEKO-TEX STANDARD 100<img alt="" src="https://m.media-amazon.com/images/I/51YvKwF01yL._SS2

Let's iterate through each product and print the brand name, which is saved as a header (h2) with a unique class in the element:

In [59]:
from tqdm import tqdm

In [60]:
# we'll populate this
data = []

xpath_brand = '//h2[@class="a-size-mini s-line-clamp-1"]'
xpath_price = '//span[@id="price-link"]'

for product in tqdm(product_tiles):
    brand = await product.locator(xpath_brand).text_content()
    price = await product.locator(xpath_price).text_content()
    row = {
        'brand' : brand,
        'price': price    
    }
    data.append(row)

 60%|████████████████████████                | 36/60 [00:30<00:20,  1.19it/s]


TimeoutError: Locator.text_content: Timeout 30000ms exceeded.
Call log:
waiting for locator("//div[@data-component-type=\"s-search-result\"]").nth(36).locator("//span[@id=\"price-link\"]")


In [61]:
data[0]

{'brand': 'Amazon Essentials', 'price': 'Price, product page'}

Although we did this all using Playwright, it's better to save the page source and then parse the saved results in BeautifulSoup, lxml, or whatever parsing software you prefer.

### Annotate the elements we find
Let's find all the ads, and highlight them red on the page.

In [62]:
# xpath_ads = '//div[@data-asin and .//a[@aria-label="View Sponsored information or leave ad feedback"]]'
xpath_ads = '//div[@data-asin and .//span[@aria-label="View Sponsored information or leave ad feedback"]]'
ads = await page.locator(xpath_ads).all()

In [63]:
len(ads)

11

You can "inject" attributes into elements, including style attributes.

In [64]:
elem = ads[0]

In [65]:
style = f"background-color: red !important; transition: all 0.5s linear;"

In [66]:
await elem.evaluate(f"el => el.setAttribute('style','{style}')")

In [67]:
async def stain(elem, color = 'red'):
    """
    Injects a style attribute to stain `elem` the `color` red.
    """
    style = f"background-color: {color} !important; "\
             "transition: all 0.5s linear;"
    await elem.evaluate(f"el => el.setAttribute('style','{style}')")

In [68]:
for elem in ads:
    await stain(elem)

### Get height of document

In [69]:
import pandas as pd

In [70]:
height = await page.evaluate("document.body.scrollHeight")

In [71]:
height

14497

Get the coorindates and size of each element using the `bounding_box` function.

In [73]:
await elem.bounding_box()

{'x': 516.8046875,
 'y': 684.984375,
 'width': 250.3984375,
 'height': 663.4921875}

In [74]:
ad_metadata = []
for elem in ads:
    if await elem.is_visible(): # use this function to only analyze visable elements
        rect = await elem.bounding_box()
        ad_metadata.append(rect)

In [75]:
df = pd.DataFrame(ad_metadata)

In [76]:
df['y']

0    -3855.968750
1    -3855.968750
2    -3855.968750
3    -1694.492188
4    -1694.492188
5    -1003.000000
6    -1003.000000
7       24.492188
8       24.492188
9      684.984375
10     684.984375
Name: y, dtype: float64

In [77]:
df['how_far_down'] = df['y'] / height

In [78]:
df.how_far_down.value_counts()

how_far_down
-0.265984    3
-0.116886    2
-0.069187    2
 0.001689    2
 0.047250    2
Name: count, dtype: int64

### Save receipts

In [79]:
# how to save what the emulator sees
source = await page.content()
with open('data/amazon_playwright_test.html', 'w') as f:
    f.write(source)

In [43]:
# just what's visible
screenshot = await page.screenshot(path='data/amazon_selenium_test.png')

### Parsing the results however you like
For me it means using lxml, but you can do this same thing in BeautifulSoup, and I encourage you do so...

In [80]:
from lxml import etree

In [81]:
dom = etree.HTML(open('data/amazon_playwright_test.html').read())

In [82]:
product_metadata = []
for result in dom.xpath('.//div[contains(@cel_widget_id, "MAIN-SEARCH_RESULTS")]'):
    # this is where you can parse as many fields as you like.
    brand, product_name = result.xpath('.//h2//text()')[:2]
    
    # ToDO get other attibutes such as ratings, price, is an ad?
    
    product_metadata.append({
        'brand': brand,
        'product_name': product_name
    })

In [83]:
pd.DataFrame(product_metadata)

,brand,product_name
0,Amazon Essentials,"Slim-Fit Long Sleeve Shirt for Men, Pocket and..."
1,CURBODO,Linen Shirts for Men Casual Short Sleeve Butto...
2,GENERIC,Jesus is Cool Cross Subtle Christian Minimal R...
3,Clothe Co.,Men's Short Sleeve Button Down Shirt with Fron...
4,Arctic Cool,Men’s Crew Neck Instant Cooling Moisture Wicki...
5,32 Degrees,Mens 4 Pack Cool Crewneck T-Shirt | Anti-Odor ...
6,Cool Tees,Funny Cat Ramen Graphic Tee Japanese Kawaii An...
7,Hanes,"Sport Men's Long-Sleeve T-Shirt Pack, Cool DRI..."
8,In my defense I was left unsupervised Shirts,In my defense I was left unsupervised Cool Fun...
9,Cool Tees,Human By Chance Alpha By Choice Cool Funny Alp...


### Automate rotating "AUTOMET"
1. Find all products
2. filter to those with brand == AUTOMET
3. Find the image
    - Inject Javascript to make it spin.

In [84]:
async def spin(elem):
    """
    Injects a style attribute to rotate `elem` 180 degrees.
    """
    style = f"transform: rotate(180deg) !important; "
    await elem.evaluate(
            f"elm => elm.setAttribute('style','{style}')"
    )

Here's how to do this for ads (which we found previously)

In [85]:
for elem in ads:
    await spin(elem)

## Here's the bountry
Ultimately, I want every product image from specific brands (of y0ur choosing) to rotate continuously.
You need a full screenshot or a video is fine, too.

Also, make sure to save the results before you parse them.

In [50]:
for product in product_tiles:
    # get the brand name...
    brand_name = await product.locator(TK)
    
    # check if brand is in list
    if brand_name in brands_to_spin:
        print(product.text_content())
        # find the image TK
        spin(product)

NameError: name 'TK' is not defined

In [ ]:
await driver.close()